In [5]:
from database import SessionLocal
from sql_alchemy_orm import User, Role  


session = SessionLocal()
# Query all users
users = session.query(User).all()
for user in users:
    print(f"{user.username} has a role of {user.role.name}")

alice123 has a role of Admin
bob456 has a role of Admin
charlie789 has a role of Editor
david000 has a role of Viewer
eve999 has a role of Viewer


In [6]:
# Query user by username
user = session.query(User).filter(User.username == 'john_doe').first()

if user:
    print(f"User found: {user.name}, {user.email}")
else:
    print("User not found.")


User not found.


### Filtering with `filter` (WHERE clause):

In [7]:
user = session.query(User).filter(User.username == 'alice123').first()
user.name

'Alice'